# Algorithm BB84 for key exchange.

The key feature of this algorithm is that Alice and Bob will share some random string by the end of the protocol.
Why would we want to do this? Well, a lot of ours communications rely on having a common secret, between the sender and the receiver, which is unknown for everyone else. And also because a lot of other quantum cryptography algorithms depend on these simple steps. This is because it exists a lot of cryptographic functions that use the same key (usually a bit string, but could be almost anything) to encrypt and decrypt messages. So you'll definitely would want a string like that when you are communicating an important message. This protocol was so important in his time because it represented the new capabilities of quantum cryptography. And as a fun fact, the name BB84 comes from the initial of his authors and the year of the discover of this protocol.
The algorithm goes as follows:

First let's define some imports.

In [2]:
%matplotlib inline
# Importing standard Qiskit libraries
from qiskit import QuantumCircuit, execute, Aer, IBMQ
from qiskit.compiler import transpile, assemble
from qiskit.tools.jupyter import *
from random import randint
import hashlib

In the next cell Alice choses a random binary string from which the Shared key will be form later. Also she selects random basis and encripts the string accordingly to the selected basis. if the basis says 1 then Alice encript the bit appliying a hadamard gate. If not, Alice leaves the bit without encrypting. This process could sound dangerous but only Alice kwnows which bits are encrypted so, anyone else will find it very difficult to know when it's time to apply a Hadamard gate. Also note that when using superposition it is almost impossible to obtain the original bits from doing just meassurements.

In [3]:
#Alice cell, Bob can't see what's going in on here
m0 = ''.join([str(randint(0,1)) for x in range(256)])
Alice_bases = [randint(0,1) for x in range(256)]
qubits = list()
for i in range(len(Alice_bases)):
    mycircuit = QuantumCircuit(1,1)
    if(Alice_bases[i] == 0):
        if(m0[i] == "1"):
            mycircuit.x(0)
    else:
        if(m0[i] == "0"):
            mycircuit.h(0)
        else:
            mycircuit.x(0)
            mycircuit.h(0)
    qubits.append(mycircuit)

In the next cell Bob measures Alice's qubits. This measurement is random because Bob doesn't know Alice's Basis. Then He receives Alice's basis. Finally, He send to Alice the index of the basis that both meassure correctly. Bob can't recover the original message from Alice because In quantum mechanics after measuring a qubit you can't recover the information of the qubit. And also, He can't have copied the qubits because you can't create a perfect copy of a qubits by the quantum mechanics laws.

In [4]:
#Bob cell, Alice can't see what's going in on here
Bob_bases = [randint(0,1) for x in range(256)]
backend = Aer.get_backend('qasm_simulator')
measurements = list()
for i in range(len(Bob_bases)):
    qubit = qubits[i]
    if(Bob_bases[i] == 0):
        qubit.measure(0,0)
    else:
        qubit.h(0)
        qubit.measure(0,0)
    result = execute(qubit, backend, shots=1, memory=True).result()
    measurements.append(int(result.get_memory()[0]))

In [5]:
#Bob cell, Alice can't see what's going in on here
I0 = list()
for i in range(len(Alice_bases)):
    if(Alice_bases[i] == Bob_bases[i]):
        I0.append(i)

In the final step, both of them take from their strings the bits which they share.

In [6]:
#Test cell to see that both of them have the same key
keyAlice = ''.join([m0[x] for x in I0])
keyBob = ''.join([str(measurements[x]) for x in I0])
print(keyAlice)
print(keyBob)
print(keyAlice == keyBob)

1010110111011110010101011000011010011011000001110101100100000001101001110001001001111000110110001000100101100111101
1010110111011110010101011000011010011011000001110101100100000001101001110001001001111000110110001000100101100111101
True


As you can see in the previous code they have a shared key.
But the important question is: why does it works?
Well it's pretty obvious that the'll share some key by the end of the protocol because they shared the index of the comon bits.
So all that is left for us to prove is "if some third party Eve, evesdrop the channel could she be able to steal the shared key?"
And the short answer is no. Let's check what tactics could try Eve:
- In the first step when Alice communicate the message, Eve only could have stolen the message. But it's encrypted so She couldn't be able to distingish anything. And aside from that She wouldn't be able to copy the message because of the properties of quantum mechanics. So in the end, the protocol have to reset itself because nobody have a shared key.
- In the second step when Alice send it's basis, Eve could have stolen Alice basis. But she didn't meassure the encrypted message because of the previous point. So in the end she could try to mimic Bob, but in the end She won't find any valuable information because She would need to communicate her basis to form an agreament whith Alice. So this tactic doesn't work.
## As you can see the protocol it's perfectly safe all because of the properties of quantum mechanics.